In [1]:
import bz2
import os

import xml.etree.ElementTree as etree

from kg.entity_linking import utils

In [2]:
wiki_dir = '/Users/tmorrill002/Documents/datasets/wikipedia/'

In [3]:
# wiki_dir = '/Users/tmorrill002/Documents/datasets/wikipedia/20210401/'
file = 'enwiki-20210401-pages-articles-multistream1.xml-p1p41242.bz2'

file_path = os.path.join(wiki_dir, file)

In [14]:
fp = bz2.BZ2File(file_path, 'r')

In [5]:
events, elements = [], []
for event, elem in etree.iterparse(fp, events=('start', 'end')):
    events.append(event)
    elements.append(elem)

In [12]:
for element in elements[97:99]:
    print(element.tag, element.text, element.attrib)

{http://www.mediawiki.org/xml/export-0.10/}timestamp 2021-01-23T15:15:01Z {}
{http://www.mediawiki.org/xml/export-0.10/}timestamp 2021-01-23T15:15:01Z {}


In [6]:
import xml.sax

class WikiXmlHandler(xml.sax.handler.ContentHandler):
    """Content handler for Wiki XML data using SAX"""
    def __init__(self):
        xml.sax.handler.ContentHandler.__init__(self)
        self._buffer = None
        self._values = {}
        self._current_tag = None
        self._pages = []

    def characters(self, content):
        """Characters between opening and closing tags"""
        if self._current_tag:
            self._buffer.append(content)

    def startElement(self, name, attrs):
        """Opening tag of element"""
        if name in ('title', 'text', 'timestamp'):
            self._current_tag = name
            self._buffer = []

    def endElement(self, name):
        """Closing tag of element"""
        if name == self._current_tag:
            self._values[name] = ' '.join(self._buffer)

        if name == 'page':
            self._pages.append((self._values['title'], self._values['text']))

In [7]:
# Content handler for Wiki XML
handler = WikiXmlHandler()

# Parsing object
parser = xml.sax.make_parser()
parser.setContentHandler(handler)

handler._pages

[]

In [16]:
counter = 0
for line in fp:
    print(line)
    if counter > 100:
        break
    # parser.feed(line)
    counter += 1

b'  <siteinfo>\n'
b'    <sitename>Wikipedia</sitename>\n'
b'    <dbname>enwiki</dbname>\n'
b'    <base>https://en.wikipedia.org/wiki/Main_Page</base>\n'
b'    <generator>MediaWiki 1.36.0-wmf.36</generator>\n'
b'    <case>first-letter</case>\n'
b'    <namespaces>\n'
b'      <namespace key="-2" case="first-letter">Media</namespace>\n'
b'      <namespace key="-1" case="first-letter">Special</namespace>\n'
b'      <namespace key="0" case="first-letter" />\n'
b'      <namespace key="1" case="first-letter">Talk</namespace>\n'
b'      <namespace key="2" case="first-letter">User</namespace>\n'
b'      <namespace key="3" case="first-letter">User talk</namespace>\n'
b'      <namespace key="4" case="first-letter">Wikipedia</namespace>\n'
b'      <namespace key="5" case="first-letter">Wikipedia talk</namespace>\n'
b'      <namespace key="6" case="first-letter">File</namespace>\n'
b'      <namespace key="7" case="first-letter">File talk</namespace>\n'
b'      <namespace key="8" case="first-letter">

In [9]:
len(handler._pages)

12

In [20]:
handler._pages[11]

('Autism',
 '{{Short description|Neurodevelopmental disorder involving social communication difficulties and repetitive behavior}} \n {{About|the classic autistic disorder|other conditions sometimes called  " autism " |Autism spectrum|the journal|Autism (journal)}} \n {{pp-semi-indef}} \n {{pp-move-indef}} \n {{Update|reason=some parts of the article do not accurately reflect either the new DSM5 or ICD10 or ICD11 criteria|date=February 2021}} \n {{bots|deny=Monkbot 18}}  < !-- keep Monkbot from visiting this page -- > \n {{Use dmy dates|date=June 2020}} \n {{Use American English|date=August 2016}} \n {{Infobox medical condition (new) \n | name         = Autism \n | image        = Autism-stacking-cans 2nd edit.jpg \n | alt          = Boy stacking cans \n | caption      = Repetitively stacking or lining up objects is commonly associated with autism. \n | field        = [[Clinical psychology]], [[psychiatry]], [[pediatrics]], [[occupational medicine]] \n | symptoms     = Trouble with [[In

In [10]:
for element in elements[:300]:
    print(element.text, element.tag, element.attrib)

NameError: name 'elements' is not defined

In [77]:
elements[85]

<Element '{http://www.mediawiki.org/xml/export-0.10/}title' at 0x114ce6b80>

In [22]:
def extract_file(self, path):
    start_time = time.time()

    count = 0
    # opens mulit-stream file
    with bz2.BZ2File(path, 'r') as fp:
        # handles first element in the iteration, which is later cleared to avoid persisting everything in memory
        # get an iterable
        context = etree.iterparse(fp, events=("start", "end"))

        # turn it into an iterator
        context = iter(context)

        # get the root element
        event, root = context.next()
        
        for event, elem in context:
            # 
            tname = utils.strip_tag_name(elem.tag)
            if event == 'start':
                if tname == 'page':
                    title = ''
                    id = -1
                    redirect = ''
                    inrevision = False
                    ns = 0
                elif tname == 'revision':
                    # Do not pick up on revision id's
                    inrevision = True
            elif event == 'end':
                # get text content that has been accumulated
                if tname == 'title':
                    title = elem.text
                elif tname == 'text':
                    text = elem.text
                elif tname == 'id' and not inrevision:
                    id = int(elem.text)
                elif tname == 'redirect':
                    redirect = elem.attrib['title']
                elif tname == 'ns':
                    ns = int(elem.text)
                elif tname == 'page':
                    self.total_count += 1

                    try:
                        if ns == 10:
                            self.template_count += 1
                            self.worker.process_template(id, title)
                        elif ns == 0:
                            if len(redirect) > 0:
                                self.article_count += 1
                                self.worker.process_redirect(
                                    id, title, redirect)
                            else:
                                self.redirect_count += 1
                                #print(f"Article: {title}")
                                self.worker.process_article(
                                    id, title, text)
                    except Exception as e:
                        print(f"Error processing: {id}:{title}")
                        print(e)

                    title = ""
                    redirect = ""
                    text = ""
                    ns = -100
                    if self.total_count > 1 and (self.total_count %
                                                 WORKER_REPORT) == 0:
                        self.worker.report_progress(self.total_count)
                        self.total_count = 0

                    root.clear()
    self.worker.report_progress(self.total_count)